In [2]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install transformers datasets torch

#mBERT/ TextFoolerJin2019.build/ arabic_preprocessed_data-3

In [4]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import textattack
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult

# Load dataset
file_path = "/content/arabic_preprocessed_data-3.csv"
df = pd.read_csv(file_path)

# Define dataset class
class ArabicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Wrap model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define attack
attack = TextFoolerJin2019.build(model_wrapper)

# Select a subset of data for attack
test_texts, test_labels = df['Text'].tolist(), df['Label'].tolist()
test_samples = list(zip(test_texts, test_labels))[:20]  # Attack first 5 samples

# Run attack
for text, label in test_samples:
    result = attack.attack(text, label)
    if isinstance(result, SuccessfulAttackResult):
        print("Original:", result.original_text)
        print("Attacked:", result.perturbed_text)
    else:
        print("Attack failed for sample:", text)


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set i

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:14<00:00, 33.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpq55zu_jo.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack failed for sample: الفيلم رائع جدا استمتعت لحظة السينما التجربة ممتعة
Attack failed for sample: الطعام بارد جدا المطعم غير نظيف اكرر الزيارة مرة اخرى
Attack failed for sample: احببت المكان جدا الناس ودودون الخدمة سريعة الطعام شهي جدا
Attack failed for sample: انصح المتجر يفتقر التنوع المنتجات اسواره مرتفعة جدا
Attack failed for sample: التجربة سيئة جدا تاخر وصول الطلب عدة ساعات الطعام غير طازج
Attack failed for sample: الفيلم مشوق جدا الاداء التمثيلي ممتاز اعجبني شيء العمل الفني
Attack failed for sample: الفندق رائع جدا الخدمة ممتازة الغرف صغيرة جدا
Attack failed for sample: الطعام المكان غير طيب الاطلاق اتمكن انهاء الوجبة
Attack failed for sample: الرحلة الجبال رائعة الطقس رائع المنظر طبيعيا جدا
Attack failed for sample: الخدمة بطيئة جدا الطعام يفتقر النكهة اعود المطعم
Attack failed for sample: سعيد جدا التجربة الجو هادئ المكان مريح جدا
Attack failed for sample: الهاتف جيدا البطارية تدوم طويلا سعره مرتفعا جدا
Attack failed for sample: الكتاب ممتازا جميع الجوانب سواء الاسلوب الم

# mBERT/ PWWSRen2019 / arabic_preprocessed_data-3

In [5]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import textattack
from textattack.attack_recipes import PWWSRen2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult

# Load dataset
file_path = "/content/arabic_preprocessed_data-3.csv"
df = pd.read_csv(file_path)

# Define dataset class
class ArabicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Wrap model for TextAttack
wrapped_model = HuggingFaceModelWrapper(model, tokenizer)

# Define attack
attack = PWWSRen2019.build(wrapped_model)


# Select a subset of data for attack
test_texts, test_labels = df['Text'].tolist(), df['Label'].tolist()
test_samples = list(zip(test_texts, test_labels))[:20]  # Attack first 5 samples

# Run attack
for text, label in test_samples:
    result = attack.attack(text, label)
    if isinstance(result, SuccessfulAttackResult):
        print("Original:", result.original_text)
        print("Attacked:", result.perturbed_text)
    else:
        print("Attack failed for sample:", text)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack failed for sample: الفيلم رائع جدا استمتعت لحظة السينما التجربة ممتعة
Attack failed for sample: الطعام بارد جدا المطعم غير نظيف اكرر الزيارة مرة اخرى
Attack failed for sample: احببت المكان جدا الناس ودودون الخدمة سريعة الطعام شهي جدا
Attack failed for sample: انصح المتجر يفتقر التنوع المنتجات اسواره مرتفعة جدا
Attack failed for sample: التجربة سيئة جدا تاخر وصول الطلب عدة ساعات الطعام غير طازج
Attack failed for sample: الفيلم مشوق جدا الاداء التمثيلي ممتاز اعجبني شيء العمل الفني
Attack failed for sample: الفندق رائع جدا الخدمة ممتازة الغرف صغيرة جدا
Attack failed for sample: الطعام المكان غير طيب الاطلاق اتمكن انهاء الوجبة
Attack failed for sample: الرحلة الجبال رائعة الطقس رائع المنظر طبيعيا جدا
Attack failed for sample: الخدمة بطيئة جدا الطعام يفتقر النكهة اعود المطعم
Attack failed for sample: سعيد جدا التجربة الجو هادئ المكان مريح جدا
Attack failed for sample: الهاتف جيدا البطارية تدوم طويلا سعره مرتفعا جدا
Attack failed for sample: الكتاب ممتازا جميع الجوانب سواء الاسلوب الم

#mBERT- TextBuggerLi2018- arabic_preprocessed_data-3

In [7]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import textattack
from textattack.attack_recipes import TextBuggerLi2018
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult

# Load dataset
file_path = "/content/arabic_preprocessed_data-3.csv"
df = pd.read_csv(file_path)

# Define dataset class
class ArabicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Wrap model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define attack
attack = TextBuggerLi2018.build(model_wrapper)

# Select a subset of data for attack
test_texts, test_labels = df['Text'].tolist(), df['Label'].tolist()
test_samples = list(zip(test_texts, test_labels))[:20]  # Attack first 5 samples

# Run attack
for text, label in test_samples:
    result = attack.attack(text, label)
    if isinstance(result, SuccessfulAttackResult):
        print("Original:", result.original_text)
        print("Attacked:", result.perturbed_text)
    else:
        print("Attack failed for sample:", text)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack failed for sample: الفيلم رائع جدا استمتعت لحظة السينما التجربة ممتعة
Attack failed for sample: الطعام بارد جدا المطعم غير نظيف اكرر الزيارة مرة اخرى
Attack failed for sample: احببت المكان جدا الناس ودودون الخدمة سريعة الطعام شهي جدا
Attack failed for sample: انصح المتجر يفتقر التنوع المنتجات اسواره مرتفعة جدا
Attack failed for sample: التجربة سيئة جدا تاخر وصول الطلب عدة ساعات الطعام غير طازج
Original: <bound method AttackResult.original_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x7cf8df85a9d0>>
Attacked: <bound method AttackResult.perturbed_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x7cf8df85a9d0>>
Original: <bound method AttackResult.original_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x7cf9035024d0>>
Attacked: <bound method AttackResult.perturbed_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResu

# mBERT/ GeneticAlgorithmAlzantot2018 / arabic_preprocessed_data-3

In [8]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import textattack
from textattack.attack_recipes import GeneticAlgorithmAlzantot2018
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult

# Load dataset
file_path = "/content/arabic_preprocessed_data-3.csv"
df = pd.read_csv(file_path)

# Define dataset class
class ArabicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Wrap model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define attack
attack = GeneticAlgorithmAlzantot2018.build(model_wrapper)

# Select a subset of data for attack
test_texts, test_labels = df['Text'].tolist(), df['Label'].tolist()
test_samples = list(zip(test_texts, test_labels))[:20]  # Attack first 5 samples

# Run attack
for text, label in test_samples:
    result = attack.attack(text, label)
    if isinstance(result, SuccessfulAttackResult):
        print(f"Original: {result.original_result.attacked_text}")
        print(f"Attacked: {result.perturbed_result.attacked_text}")
    else:
        print(f"Attack failed for sample: {text}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
textattack: Downloading https://textattack.s3.amazonaws.com/constraints/semantics/language-models/alzantot-goog-lm.
100%|██████████| 3.87G/3.87G [01:45<00:00, 36.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpyc9s_3ce.zip to /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm.
textattack: Successfully saved constraints/semantics/language-models/alzantot-goog-lm to cache.
Recovering graph.
INFO:tensorflow:Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt
Recovering checkpoint /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/ckpt-*
textattack: Unkn

Attack failed for sample: الفيلم رائع جدا استمتعت لحظة السينما التجربة ممتعة
Attack failed for sample: الطعام بارد جدا المطعم غير نظيف اكرر الزيارة مرة اخرى
Attack failed for sample: احببت المكان جدا الناس ودودون الخدمة سريعة الطعام شهي جدا
Attack failed for sample: انصح المتجر يفتقر التنوع المنتجات اسواره مرتفعة جدا
Attack failed for sample: التجربة سيئة جدا تاخر وصول الطلب عدة ساعات الطعام غير طازج
Attack failed for sample: الفيلم مشوق جدا الاداء التمثيلي ممتاز اعجبني شيء العمل الفني
Attack failed for sample: الفندق رائع جدا الخدمة ممتازة الغرف صغيرة جدا
Attack failed for sample: الطعام المكان غير طيب الاطلاق اتمكن انهاء الوجبة
Attack failed for sample: الرحلة الجبال رائعة الطقس رائع المنظر طبيعيا جدا
Attack failed for sample: الخدمة بطيئة جدا الطعام يفتقر النكهة اعود المطعم
Attack failed for sample: سعيد جدا التجربة الجو هادئ المكان مريح جدا
Attack failed for sample: الهاتف جيدا البطارية تدوم طويلا سعره مرتفعا جدا
Attack failed for sample: الكتاب ممتازا جميع الجوانب سواء الاسلوب الم

# mBERT -BERT-Attack- arabic_preprocessed_data-3

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import textattack
from textattack.attack_recipes import BERTAttackLi2020
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult

# Load dataset
file_path = "/content/arabic_preprocessed_data-3.csv"
df = pd.read_csv(file_path)

# Define dataset class
class ArabicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Wrap model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define attack
attack = BERTAttackLi2020.build(model_wrapper)

# Select a subset of data for attack
test_texts, test_labels = df['Text'].tolist(), df['Label'].tolist()
test_samples = list(zip(test_texts, test_labels))[:20]  # Attack first 5 samples

# Run attack
for text, label in test_samples:
    result = attack.attack(text, label)
    if isinstance(result, SuccessfulAttackResult):
        print(f"Original: {result.original_result.attacked_text}")
        print(f"Attacked: {result.perturbed_result.attacked_text}")
    else:
        print(f"Attack failed for sample: {text}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.5

In [11]:
!textattack attack --model-from-huggingface bert-base-multilingual-cased --dataset-from-file arabic_preprocessed_data-3.py --recipe deepwordbug --num-examples 20


2025-02-24 09:58:40.916185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740391121.260563   16843 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740391121.335967   16843 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
textattack: Loading model and tokenizer from file: None
textattack: Loading module from `arabic_preprocessed_data-3.py`.
textattack: Loading pre-trained model from HuggingFace model repository: bert-base-multilingual-cased
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably T